In [2]:
import numpy as np
import pandas as pd

In [3]:
class conjugateGradient():
    def __init__(self, A, b, c, x0):
        self.A = A
        self.b = b
        self.x = x0
        self.c = c
        self.X, self.P, self.NormG, self.funValue = [], [], [], []
    # 定义二次函数
    def fun(self, x):
        return (x.T)@self.A@x/2 + self.b@x + self.c
    
    # 梯度
    def objGradient(self, x):
        g = self.A @ np.array(x) + self.b
        gNorm = np.linalg.norm(g, axis = 0)
        return np.array(g), gNorm
    
    # 迭代主体
    def iterate(self, epsilon, t0 = 0.5, h0 = 0.1, alpha = 1.1, epsilon_LS = 0.001):
        '''
        params:
        epsilon:迭代终止界限，梯度范数小于epsilon
        t0:每次区间探索的初始值
        h0:每次区间探索的初始探索步长
        alpha:加步系数
        '''
        self.X, self.P, self.NormG, self.funValue = [], [], [], []
        x = self.x
        self.X.append(x)
        g, gNorm = self.objGradient(x)
        self.funValue.append(self.fun(x))# 记录迭代函数值
        while gNorm > epsilon:
            x, gNorm = self.conjungate(x, epsilon, t0, h0, alpha, epsilon_LS)
        return self.X, self.P, self.NormG, self.funValue
    
    def conjungate(self, initial_x, epsilon, t0, h0, alpha, epsilon_LS):
        g0, gNorm = self.objGradient(initial_x)
        xk = initial_x
        p0 = -g0
        for k in range(2):
            if gNorm <= epsilon:
                return xk, gNorm
            else:
                self.NormG.append(gNorm)
                self.P.append(p0)
                interval = self.searchInterval(t0, h0, alpha, xk, -p0)
                tk = self.goldenSplit(xk, interval, epsilon_LS, -p0)
                xk = xk + tk*p0
                self.X.append(xk)
                g1, g1Norm = self.objGradient(xk)
                self.funValue.append(self.fun(xk))
                p0 = -g1 + (g1Norm**2)*(p0)/(gNorm**2)
                gNorm = g1Norm
        self.NormG.append(g1Norm)
        self.P.append(p0)
        return xk, gNorm
    def goldenSplit(self, x, t, epsilon, direction):
        '''
        黄金分割法
        param
        t:搜索区间
        direction:搜索方向
        return：搜索结果t
        '''
        a, b = t[0], t[1]
        t1 = a + 0.618*(b-a)
        t2 = a + 0.382*(b-a)
        fi1, fi2 = self.fun(x-t1*direction), self.fun(x-t2*direction)
        while abs(t2 - t1) >= epsilon:
            if fi1 < fi2:
                a = t2
                t2, fi2 = t1, fi1
                t1 = a + b - t2
                fi1 = self.fun(x-t1*direction)
            else:
                b = t1
                t1, fi1 = t2, fi2
                t2 = a + 0.382*(b-a)
                fi2 = self.fun(x-t2*direction)
        return (t1 + t2)/2
    def searchInterval(self, t0, h0, alpha, x, direction):
        '''
        确定搜索区间
        '''
        k, t, h= 0, t0, h0
        tk = t
        tk_1 = tk + h
        fik, fik_1 = self.fun(x-tk*direction), self.fun(x-tk_1*direction)
        if fik > fik_1:
            h = alpha*h
            t, tk = tk, tk_1
            fik = fik_1
            tk_1 = tk + h
            fik_1 = self.fun(x-tk_1*direction)
            while fik > fik_1:
                h = alpha*h
                t, tk = tk, tk_1
                fik = fik_1
                tk_1 = tk + h
                fik_1 = self.fun(x-tk_1*direction)
        else:
            h = -h
            t = tk_1
            tk_1 = tk + h
            fik_1 = self.fun(x-fik_1*direction)
            while fik > fik_1:
                h = alpha*h
                t, tk = tk, tk_1
                fik = fik_1
                tk_1 = tk + h
                fik_1 = self.fun(x-tk_1*direction)
        return [min([t, tk_1]), max([t, tk_1])]

In [4]:
A = np.array([[2, 0], [0, 8]])
b = np.array([0, 0])
c = 0
epsilon = 0.01
x0 = np.array([1, 1])
fourth = conjugateGradient(A, b, c, x0)

In [5]:
X, P, NormG, funValue = fourth.iterate(0.01, t0=0.1, h0 = 0.05, epsilon_LS = 0.0001)
res4 = pd.DataFrame({'迭代x值':X, 'P':P, '梯度范数':NormG, '函数值：':funValue})

In [6]:
res4

,迭代x值,P,梯度范数,函数值：
0,"[1, 1]","[-2, -8]",8.246211,5.000000
1,"[0.7384119275411292, -0.04635228983548334]","[-1.5450154705061276, 0.09805185698838992]",1.522667,0.553846
2,"[0.0001421289205489895, 0.0005007845155967222]","[-0.00029500727511982456, -0.004005593929684126]",0.004016,0.000001


In [7]:
A = np.array([[4, -1], [-1, 2]])
b = np.array([0, 0])
c = 0
epsilon = 0.01
x0 = np.array([1, 1])
fifth = conjugateGradient(A, b, c, x0)

In [8]:
X, P, NormG, funValue = fifth.iterate(epsilon=epsilon, t0=0.1, h0 = 0.05, epsilon_LS = 0.0001)
res5 = pd.DataFrame({'迭代x值':X, 'P':P, '梯度范数':NormG, '函数值：':funValue})

In [9]:
res5

,迭代x值,P,梯度范数,函数值：
0,"[1, 1]","[-3, -1]",3.162278,2.000000e+00
1,"[0.0626670889009503, 0.6875556963003168]","[-0.1371268389358703, -1.5037823635771452]",1.383250,4.375000e-01
2,"[-1.1032684579068142e-05, 0.00020481332296873322]","[0.0002489269379998704, -0.0004208471106359698]",0.000489,4.445158e-08
